In [362]:
pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [363]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

df = pd.read_csv('./dataset/2006_2014_ver1.csv')
                              # 연속변수 타겟 VALP를 가지고 스케일 조정이 되지 않은 데이터셋인 house-unscaled-VALP.csv 불러오기
df.shape

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20084\2680870450.py:8: DtypeWarning: Columns (4,6,7,8,9,10,11,12,13,14,15,16,17,18,20,21,22,24,25,26,27,28,29,30,32,33,34,35,36,37,38,40,41,42,44,45,46,47,48,49,50,52,53,54,56,57,58,59,60,61,62,64,65,66,68,69,70,72,73,74,76,77,78,79,80,81,83,84,86,89,102,103,104,105,106,107,109,110,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,216,218,219,220,221,223,224,225,226,227,228,229,233,234,235,236,239,243,244,245,246,247,248,249,250,251,252) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/2006_2014_ver1.csv')


(102773, 253)

In [364]:
df = df[df['Year']==2013 ]
df = df[df['CompanyID'].notna() & (df['CompanyID'] != '*')]

In [365]:
# 타겟 분리
target=df['EBizSystem2']
# 'biz' 단어가 들어간 열을 찾고 해당 열을 삭제합니다.
columns_to_drop = df.columns[df.columns.str.contains('Biz', case=False)]
df = df.drop(columns=columns_to_drop)

print(df)

       Year  Area IndCategory1  IndCategory2   CompanyID CapitalRatio1  \
78124  2013    11            A             3  2376132235             0   
78125  2013    11            A             3  4996132235             0   
78126  2013    11            A             3  9020122135             0   
78127  2013    11            A             3  9173122135             0   
78129  2013    11            B             7  4436132235             0   
...     ...   ...          ...           ...         ...           ...   
90351  2013    39            R            91  5593122032             0   
90352  2013    39            R            91  6256152135             0   
90353  2013    39            R            91  6592192934             0   
90354  2013    39            R            91  7672162534             0   
90355  2013    39            R            91  8265182037           100   

      StockMktListing Emp_total Emp_male Emp_female  ...  Cost8 Cost9 Cost10  \
78124               1       219

In [366]:
# 결측값이 50% 이상인 열을 찾고 해당 열을 삭제합니다.

threshold = 0.5
missing_ratios = df.isnull().mean()
columns_to_drop = missing_ratios[missing_ratios > threshold].index.tolist()
df = df.drop(columns=columns_to_drop)

print(df)

       Year  Area IndCategory1  IndCategory2   CompanyID CapitalRatio1  \
78124  2013    11            A             3  2376132235             0   
78125  2013    11            A             3  4996132235             0   
78126  2013    11            A             3  9020122135             0   
78127  2013    11            A             3  9173122135             0   
78129  2013    11            B             7  4436132235             0   
...     ...   ...          ...           ...         ...           ...   
90351  2013    39            R            91  5593122032             0   
90352  2013    39            R            91  6256152135             0   
90353  2013    39            R            91  6592192934             0   
90354  2013    39            R            91  7672162534             0   
90355  2013    39            R            91  8265182037           100   

      StockMktListing Emp_total Emp_male Emp_female  ...  Cost8 Cost9 Cost10  \
78124               1       219

In [367]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11887 entries, 78124 to 90355
Columns: 166 entries, Year to Cost17
dtypes: int64(3), object(163)
memory usage: 15.1+ MB


In [368]:
df.columns

Index(['Year', 'Area', 'IndCategory1', 'IndCategory2', 'CompanyID',
       'CapitalRatio1', 'StockMktListing', 'Emp_total', 'Emp_male',
       'Emp_female',
       ...
       'Cost8', 'Cost9', 'Cost10', 'Cost11', 'Cost12', 'Cost13', 'Cost14',
       'Cost15', 'Cost16', 'Cost17'],
      dtype='object', length=166)

In [369]:
df=df.replace('*',np.nan)

In [370]:
outsourcing_cols = df.columns[df.columns.str.contains('Out', case=False)]

print(outsourcing_cols)

Index(['OutsourcingCost'], dtype='object')


In [371]:
df_cat = df[['Year','Area','IndCategory1','IndCategory2','CompanyID','StockMktListing','Subsidiary1','ParentCompany1','Franchise1','Overseas1','NewEntry3','SAlliance1']]

df_num = df.drop(columns=df_cat.columns)
df_num= df_num.replace(np.nan,0)

In [372]:
# 범주형 연속형 분리 -> 더미화, data scaling, 상관계수 0.7 이상 제거
# 더미화가... 안된건가..????   저 숫자가 뭘 의미하는지 애매하다 > 섣불리 전부 더미화를 해야할지 모르겠음
df_cat = df_cat.astype('object')
df_num = df_num.astype('int')

In [373]:
pd.concat([df_cat, target], axis=1)

,Year,Area,IndCategory1,IndCategory2,CompanyID,StockMktListing,Subsidiary1,ParentCompany1,Franchise1,Overseas1,NewEntry3,SAlliance1,EBizSystem2
78124,2013,11,A,3,2376132235,1,1,2,2,1,2,2,1
78125,2013,11,A,3,4996132235,1,1,2,2,1,2,2,1
78126,2013,11,A,3,9020122135,3,1,2,2,1,2,2,1
78127,2013,11,A,3,9173122135,3,2,2,2,2,2,2,1
78129,2013,11,B,7,4436132235,3,1,2,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90351,2013,39,R,91,5593122032,3,2,2,2,2,1,2,NaN
90352,2013,39,R,91,6256152135,3,2,1,2,2,2,2,NaN
90353,2013,39,R,91,6592192934,3,2,2,2,2,2,2,NaN
90354,2013,39,R,91,7672162534,3,2,2,2,2,2,1,NaN


In [374]:
# 인사이트
# 데이터 분석시에는 관련성이 없는 companyid 를 제거해야함. year도 고정이므로 필요없음. 
# IndCategory1 , 2 는 거의 100프로 일치. 공선성으로 인해 제거해야함

In [375]:
from sklearn.preprocessing import StandardScaler

# StandardScaler객체 생성
scaler = StandardScaler()
# StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
df_num_scaler = scaler.fit_transform(df_num)

#transform( )시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
df_num_scaler = pd.DataFrame(data=df_num_scaler)

In [376]:
df_num_scaler.corr()

,0,1,2,3,4,5,6,7,8,9,...,144,145,146,147,148,149,150,151,152,153
0,1.000000,0.013094,0.008671,0.020261,-0.002334,0.024314,0.022146,0.021193,0.011213,0.033931,...,0.010548,0.030458,0.001276,0.015175,0.033999,-0.002785,-0.000423,0.000476,-0.012368,-0.002518
1,0.013094,1.000000,0.969371,0.804382,0.410390,0.513056,0.486882,0.407526,0.148703,0.544479,...,0.949981,0.340356,0.782773,0.607365,0.727247,0.251835,0.708269,0.591121,0.457190,0.720236
2,0.008671,0.969371,1.000000,0.633829,0.379162,0.474223,0.489269,0.297169,0.156585,0.522673,...,0.958492,0.291807,0.760270,0.549861,0.706383,0.204180,0.698724,0.603805,0.469369,0.701187
3,0.020261,0.804382,0.633829,1.000000,0.375245,0.468615,0.349792,0.564565,0.089530,0.450376,...,0.673173,0.365999,0.626085,0.582655,0.581572,0.299188,0.540331,0.401008,0.304421,0.572060
4,-0.002334,0.410390,0.379162,0.375245,1.000000,0.123871,0.122397,0.088573,0.381849,0.183745,...,0.469083,0.199119,0.474745,0.214033,0.378246,0.127347,0.519719,0.382090,0.349868,0.446009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,-0.002785,0.251835,0.204180,0.299188,0.127347,0.172395,0.156922,0.150469,0.031573,0.311708,...,0.271472,0.179256,0.101484,0.323783,0.142006,1.000000,0.123083,0.129856,0.078334,0.165669
150,-0.000423,0.708269,0.698724,0.540331,0.519719,0.180654,0.187193,0.111570,0.172240,0.230646,...,0.815971,0.166920,0.814396,0.387949,0.747474,0.123083,1.000000,0.598452,0.309444,0.874193
151,0.000476,0.591121,0.603805,0.401008,0.382090,0.345032,0.366205,0.195489,0.141340,0.348380,...,0.618834,0.271078,0.599764,0.370413,0.482339,0.129856,0.598452,1.000000,0.599921,0.566253
152,-0.012368,0.457190,0.469369,0.304421,0.349868,0.302661,0.312775,0.188623,0.068517,0.371445,...,0.409951,0.447311,0.487203,0.426170,0.311847,0.078334,0.309444,0.599921,1.000000,0.259884


In [ ]:
from imblearn.over_sampling import SMOTE
# over-sampling 설정 ( 1 의 비율이 9.2%로 매우 적음)
sm = SMOTE(sampling_strategy='auto')
# train데이터를 이용한 Over-sampling
x_resampled,y_resampled = sm.fit_resample(df_train_x,df_train_y)
#결과 확인
print('Over-Sampling 전:\n',df_train_y.value_counts(),"\n")
print('Over-Sampling 후 Train X: {}'.format(x_resampled.shape))
print('Over-Sampling 후 Train Y: {}\n'.format(y_resampled.shape))
print("Over-Sampling 후 '1':{}".format(sum(y_resampled==1)))
print("Over-Sampling 후 '0':{}".format(sum(y_resampled==0)))

In [ ]:
target.value_counts(10)

In [ ]:
data = df.drop(['VALP'], axis=1)   # 타겟변수(VALP임에 주의)를 제외한 입력변수를 data에 저장
target = df['VALP']                # 타겟변수만 target에 저장

# 50:50 데이터 변환
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( 
    data, target, test_size=0.5, random_state=42)   # test_size=0.5임에 주의 

# 6.4.3 LightGBM 모델

In [ ]:
# 기본 lGBMRegressor 모델
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

lgb = LGBMRegressor(random_state=0)
lgb.fit(X_train, y_train)
pred = lgb.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

In [ ]:
#그리드 서치 실행
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lgb = LGBMRegressor()

parameters = {'colsample_bytree': [1.0,0.4,2.7],
              'learning_rate': [0.7,0.9, 0.2],
              'max_depth': [14],
              'min_child_weight' : [9],
              'n_estimators': [1000],
              'subsample': [0.3, 0.6]
              }

lgb_grid = GridSearchCV(lgb,
                        parameters,
                        scoring = 'r2',
                        cv = 3,
                        n_jobs = -1,
                        verbose=True)
lgb_grid.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', lgb_grid.best_params_)

In [ ]:
from sklearn.metrics import r2_score

model = lgb_grid.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

In [ ]:
from lightgbm import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(8, 10))
plot_importance(model, ax=ax);

# 6.5 스태킹 모델(앙상블)

In [ ]:
model_xgb = xgb_grid.best_estimator_
pred_xgb = model_xgb.predict(X_test)

model_lgb = lgb_grid.best_estimator_
pred_lgb = model_lgb.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score

pred = 0.5*pred_xgb + 0.5*pred_lgb
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

In [ ]:
# 참조 코딩
pred = 0.6*pred_xgb + 0.4*pred_lgb
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

In [ ]:
# 참조 코딩
pred = 0.7*pred_xgb + 0.3*pred_lgb
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

In [ ]:
# 참조 코딩
pred = 0.8*pred_xgb + 0.2*pred_lgb
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

In [ ]:
pred = 0.9*pred_xgb + 0.1*pred_lgb
print('r2: {0:.5f}'.format(r2_score(y_test, pred)))